Produce plots for the third experimental section (models trained on real data)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import glob
import wandb

sns.set()

# matplotlib.use("pgf")
matplotlib.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "font.serif": "Palatino",
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)

plot_dir = "/Users/vaclav/prog/thesis/figures/"

In [ ]:
models = [
    # default params
    {"dataset": "Violin", "name": "0725-ddspae-cnn-1", "display_name": "DDSP-CNN"},
    {"dataset": "Trumpet", "name": "0804-ddspae-cnn-3", "display_name": "DDSP-CNN"},
    # ch=16
    {"dataset": "Violin", "name": "0809-ddspae-cnn-5", "display_name": "DDSP-CNN-Tiny"},
    {"dataset": "Trumpet", "name": "0809-ddspae-cnn-4", "display_name": "DDSP-CNN-Tiny"},
    #     # inverted bottleneck, ch=32
    {"dataset": "Violin", "name": "0726-ddspae-cnn", "display_name": "DDSP-CNN-IB"},
    {"dataset": "Trumpet", "name": "0809-ddspae-cnn", "display_name": "DDSP-CNN-IB"},
]

models2 = []
for m in models:
    m = m.copy()
    m["quantization"] = False
    models2.append(m)
    m = m.copy()
    m["quantization"] = True
    models2.append(m)

models = models2

eval_dir = "/Users/vaclav/prog/thesis/data/eval_data/"

api = wandb.Api()

for model in models:
    model_name = model["name"]

    suffix = "rtq" if model["quantization"] else "rt"
    model_eval_name = f"{model_name}-{suffix}"

    artifact = api.artifact(
        f"neural-audio-synthesis-thesis/nas-evaluation/eval-{model_eval_name}:latest"
    )
    csv_dir = artifact.checkout()

    run = artifact.logged_by()

    model["csv_path"] = os.path.join(csv_dir, f"eval-{model_eval_name}.csv")

    model["loss"] = run.summary["losses/total_loss"]
    
    df = pd.read_csv(model["csv_path"], index_col=0)
    model["decoder_inference_time"] = df["Autoencoder.decoder"].mean()
    model["decoder_inference_times"] = df["Autoencoder.decoder"]
    
    std = model["decoder_inference_times"].std()
    print(f"standard deviation: {std:.05f}, relative {std/model['decoder_inference_time']:.3f}")

    print(artifact.created_at, model_eval_name)

In [ ]:
df_times = []

for model in models:
    cur = pd.DataFrame(
        model["decoder_inference_times"].copy() * 1000
    )
    cur.columns = ["inference_time_ms"]
    cur["display_name"] = model["display_name"]
    cur["quantization"] = model["quantization"]
    cur["dataset"] = model["dataset"]
    df_times.append(cur)

df_times = pd.concat(df_times)
df_times

In [ ]:
df = pd.DataFrame(
    [
        [
            m["display_name"],
            m["quantization"],
            m["dataset"],
            m["loss"],
            m["decoder_inference_time"] * 1000,
        ]
        for m in models
    ],
    columns=["Model", "Quantization", "Dataset", "Loss", "Decoder inference time (ms)"],
)
df_agg = df.pivot(index=["Model", "Dataset"], columns="Quantization")
# df.columns = df.columns.get_level_values(1)
# df.columns.name = None
# df = df.reset_index()

In [ ]:
df_agg["Loss"]

In [ ]:
df_latex_table

In [ ]:
df_latex_table = df_agg["Loss"].copy()
df_latex_table.columns = ["Unquantized", "Quantized"]
# df_latex_table.columns.name = "Quantization"
# df_latex_table.index.name = "Runtime"
df_latex_table.columns.name = None
df_latex_table.index.name = None

styler = (
    df_latex_table.style
        .format_index(escape="latex")
        .format(precision=2, na_rep="---")
)

print(styler.to_latex(hrules=True))

In [ ]:
df_agg["Decoder inference time (ms)"].mean(axis="columns")

In [ ]:
df_times

In [ ]:
pdf_metadata = None
if pdf_metadata is None:
    pdf_metadata = {}

ci = 95
g = sns.catplot(
    data=df_times,
    kind="bar",
    y="display_name",
    x="inference_time_ms",
    hue="quantization",
    ci=ci,
    palette="dark",
    alpha=0.6,
    height=3,
    aspect=3 / 2,
)
plt.xlabel("Inference time (ms)")
plt.ylabel("Model")

# plt.title()

pdf_metadata = {"models": [x["name"] for x in models]}
pdf_metadata["ci"] = ci

g.fig.savefig(
    os.path.join(plot_dir, f"inference-time-optimized.pdf"),
    bbox_inches="tight",
    # Metadata doc: https://matplotlib.org/stable/api/backend_pdf_api.html#matplotlib.backends.backend_pdf.PdfFile
    metadata={"Title": " ".join([f"{k}={v}" for k, v in pdf_metadata.items()])},
    backend="pgf",
)

# print(make_table(df))

In [ ]:
models[2]["decoder_inference_times"]